In [1]:
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

from env import get_db_url
import acquire


In [2]:
query = '''
select * 
from `predictions_2017`
left join properties_2017
using(`parcelid`)
left join `airconditioningtype`
using (`airconditioningtypeid`)
left join `architecturalstyletype` as arch
using(`architecturalstyletypeid`)
left join `buildingclasstype`
using(`buildingclasstypeid`)
left join `heatingorsystemtype`
using(`heatingorsystemtypeid`)
left join `propertylandusetype`
using(`propertylandusetypeid`)
left join `storytype`
using(`storytypeid`)
left join `typeconstructiontype`
using(`typeconstructiontypeid`)
where (`latitude` is not null) and (`longitude` is not NULL)
'''
df = pd.read_sql(query, get_db_url('zillow'))

In [3]:
zillow = df.sort_values(by = ['transactiondate'], axis = 0).drop_duplicates(keep = 'last', subset = 'parcelid')
zillow.shape

(77381, 69)

In [4]:
zillow.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
typeconstructiontypeid,222.00,6.04,0.56,4.00,6.00,6.00,6.00,13.00
storytypeid,50.00,7.00,0.00,7.00,7.00,7.00,7.00,7.00
propertylandusetypeid,"77,381.00",261.83,5.14,31.00,261.00,261.00,266.00,275.00
heatingorsystemtypeid,"49,440.00",3.92,3.59,1.00,2.00,2.00,7.00,24.00
buildingclasstypeid,15.00,3.93,0.26,3.00,4.00,4.00,4.00,4.00
architecturalstyletypeid,206.00,7.39,2.73,2.00,7.00,7.00,7.00,21.00
airconditioningtypeid,"24,953.00",1.81,2.97,1.00,1.00,1.00,1.00,13.00
parcelid,"77,381.00","13,007,150.51","3,481,345.65","10,711,855.00","11,538,305.00","12,531,568.00","14,211,831.00","167,689,317.00"
id,"77,381.00","38,848.69","22,402.43",0.00,"19,461.00","38,870.00","58,253.00","77,613.00"
logerror,"77,381.00",0.02,0.17,-4.66,-0.02,0.01,0.04,5.26


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77580 entries, 0 to 77579
Data columns (total 69 columns):
typeconstructiontypeid          223 non-null float64
storytypeid                     50 non-null float64
propertylandusetypeid           77580 non-null float64
heatingorsystemtypeid           49572 non-null float64
buildingclasstypeid             15 non-null float64
architecturalstyletypeid        207 non-null float64
airconditioningtypeid           25007 non-null float64
parcelid                        77580 non-null int64
id                              77580 non-null int64
logerror                        77580 non-null float64
transactiondate                 77580 non-null object
id                              77580 non-null int64
basementsqft                    50 non-null float64
bathroomcnt                     77580 non-null float64
bedroomcnt                      77580 non-null float64
buildingqualitytypeid           49810 non-null float64
calculatedbathnbr              

In [6]:
# find nulls by collumns
count_missing = zillow.isnull().sum()
average_missing = count_missing/zillow.shape[0]
missing_cols = pd.DataFrame({'num_rows_missing': count_missing, 'pct_rows_missing': average_missing})
missing_cols

,num_rows_missing,pct_rows_missing
typeconstructiontypeid,77159,1.00
storytypeid,77331,1.00
propertylandusetypeid,0,0.00
heatingorsystemtypeid,27941,0.36
buildingclasstypeid,77366,1.00
architecturalstyletypeid,77175,1.00
airconditioningtypeid,52428,0.68
parcelid,0,0.00
id,0,0.00
logerror,0,0.00


better to just drop collumns with more than 75% info missing

In [16]:
features = missing_cols[missing_cols.pct_rows_missing < .75].index.tolist()
features

['propertylandusetypeid',
 'heatingorsystemtypeid',
 'airconditioningtypeid',
 'parcelid',
 'id',
 'logerror',
 'transactiondate',
 'id',
 'bathroomcnt',
 'bedroomcnt',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'fips',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'propertycountylandusecode',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'roomcnt',
 'unitcnt',
 'yearbuilt',
 'structuretaxvaluedollarcnt',
 'taxvaluedollarcnt',
 'assessmentyear',
 'landtaxvaluedollarcnt',
 'taxamount',
 'censustractandblock',
 'airconditioningdesc',
 'heatingorsystemdesc',
 'propertylandusedesc']

In [21]:
num_cols_missing = zillow.isnull().sum(axis = 1)
pct_cols_missing = num_cols_missing/zillow.shape[1]
num_rows = pd.DataFrame({'num_rows':num_cols_missing}).groupby(num_cols_missing).count()
num_pct = pd.DataFrame({'num_cols_missing': num_cols_missing, 'pct_cols_missing': pct_cols_missing})
missing_stats = pd.merge(num_pct, num_rows, left_on='num_cols_missing', right_on=num_rows.index).drop_duplicates()
missing_stats = missing_stats.sort_values('num_cols_missing').reset_index(drop = True)
missing_stats

,num_cols_missing,pct_cols_missing,num_rows
0,23,0.33,2
1,24,0.35,13
2,25,0.36,24
3,26,0.38,65
4,27,0.39,316
5,28,0.41,455
6,29,0.42,5270
7,30,0.43,3455
8,31,0.45,9891
9,32,0.46,12579


In [20]:
acquire.nulls_by_row(zillow[features])

,num_cols_missing,pct_cols_missing,num_rows
0,2,0.05,10428
1,3,0.07,10387
2,4,0.10,13972
3,5,0.12,13258
4,6,0.14,2627
5,7,0.17,4657
6,8,0.19,15917
7,9,0.21,4764
8,10,0.24,698
9,11,0.26,383


In [25]:
features.remove('id')
features

['propertylandusetypeid',
 'heatingorsystemtypeid',
 'airconditioningtypeid',
 'parcelid',
 'logerror',
 'transactiondate',
 'bathroomcnt',
 'bedroomcnt',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'fips',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'propertycountylandusecode',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'roomcnt',
 'unitcnt',
 'yearbuilt',
 'structuretaxvaluedollarcnt',
 'taxvaluedollarcnt',
 'assessmentyear',
 'landtaxvaluedollarcnt',
 'taxamount',
 'censustractandblock',
 'airconditioningdesc',
 'heatingorsystemdesc',
 'propertylandusedesc']